# Equation 4 Hamiltonian - ZXW Diagram and Eigenvalues

This notebook creates a ZXW diagram representing equation 4 from paper 2408:
**Ĥ^jk = F_jk σ̂_+^j σ̂_-^k + F_kj σ̂_-^j σ̂_+^k**

Which decomposes into Pauli strings as: **(F_jk/2) * (X_j X_k - Y_j Y_k)**

We then compute the eigenvalues of this Hamiltonian using the ZXW method.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import pyzx as zx
# Import ZXW functions
from pauli_hamiltonian_zx import PauliHamiltonianZX


from cor_decay_zxw import (
    compute_F_jk_equation5,
    create_equation4_hamiltonian,
    setup_positions_3d_grid,
    setup_positions_2d_grid
)


## Setup: Atom Positions and F_jk Matrix


In [2]:
# Parameters
N = 4 # Number of atoms (small for visualization)
lambda_val = 1.0  # Wavelength
gam = 1.0  # Decay rate Γ
m = 1.5  # Spacing multiplier

print(f"Setting up {N} atoms in 3D grid")
print(f"Wavelength: {lambda_val}, Decay rate: {gam}")

# Setup 3D grid positions
# x, y, z = setup_positions_3d_grid(N, m, lambda_val)
two_dim_positions = setup_positions_2d_grid(N, m, lambda_val)
print(two_dim_positions)
for i in range(len(two_dim_positions[0])):
    print(f"  Atom {i}: ({two_dim_positions[0][i]:.3f}, {two_dim_positions[1][i]:.3f})")

#for i in range(N):
#    print(f"  Atom {i}: ({x[i]:.3f}, {y[i]:.3f}, {z[i]:.3f})")


Setting up 4 atoms in 3D grid
Wavelength: 1.0, Decay rate: 1.0
(array([0.  , 0.  , 0.75, 0.75]), array([0.  , 0.75, 0.  , 0.75]), array([0., 0., 0., 0.]))
  Atom 0: (0.000, 0.000)
  Atom 1: (0.000, 0.750)
  Atom 2: (0.750, 0.000)
  Atom 3: (0.750, 0.750)


In [3]:
# Compute F_jk matrix using equation 5
F_matrix = compute_F_jk_equation5(*two_dim_positions, lambda_val, gam)

print(f"F_jk matrix shape: {F_matrix.shape}")
print(f"\nF_jk matrix (real part):")
print(F_matrix)

print(f"\nSample coupling values:")
for j in range(min(3, N)):
    for k in range(j+1, min(3, N)):
        print(f"  F_{j}{k} = {F_matrix[j, k].real:.6f} + {F_matrix[j, k].imag:.6f}i")


F_jk matrix shape: (4, 4)

F_jk matrix (real part):
[[1.-1.j         0.+0.15198794j 0.+0.15198794j 0.-0.05659477j]
 [0.+0.15198794j 1.-1.j         0.-0.05659477j 0.+0.15198794j]
 [0.+0.15198794j 0.-0.05659477j 1.-1.j         0.+0.15198794j]
 [0.-0.05659477j 0.+0.15198794j 0.+0.15198794j 1.-1.j        ]]

Sample coupling values:
  F_01 = 0.000000 + 0.151988i
  F_02 = 0.000000 + 0.151988i
  F_12 = 0.000000 + -0.056595i


In [4]:
# Create Pauli string Hamiltonian for equation 4
# This uses: (F_jk/2) * (X_j X_k - Y_j Y_k)
pauli_strings = create_equation4_hamiltonian(N, F_matrix)

print(f"Number of Pauli terms: {len(pauli_strings)}")
print(f"\nFirst 6 Pauli strings (showing X_j X_k and -Y_j Y_k pairs):")
for i, (coeff, gates) in enumerate(pauli_strings[:]):
    print(f"  {i+1}: {coeff:.6f} * {gates}")

# Create ZXW Hamiltonian
hamiltonian = PauliHamiltonianZX(pauli_strings)
print(f"\nTotal qubits: {hamiltonian.total_qubits}")


Number of Pauli terms: 16

First 6 Pauli strings (showing X_j X_k and -Y_j Y_k pairs):
  1: 0.500000-0.500000j * ['Z0']
  2: 0.000000+0.075994j * ['X0', 'X1']
  3: -0.000000-0.075994j * ['Y0', 'Y1']
  4: 0.000000+0.075994j * ['X0', 'X2']
  5: -0.000000-0.075994j * ['Y0', 'Y2']
  6: 0.000000-0.028297j * ['X0', 'X3']
  7: 0.000000+0.028297j * ['Y0', 'Y3']
  8: 0.500000-0.500000j * ['Z1']
  9: 0.000000-0.028297j * ['X1', 'X2']
  10: 0.000000+0.028297j * ['Y1', 'Y2']
  11: 0.000000+0.075994j * ['X1', 'X3']
  12: -0.000000-0.075994j * ['Y1', 'Y3']
  13: 0.500000-0.500000j * ['Z2']
  14: 0.000000+0.075994j * ['X2', 'X3']
  15: -0.000000-0.075994j * ['Y2', 'Y3']
  16: 0.500000-0.500000j * ['Z3']

Total qubits: 4


In [5]:
# Build the ZXW diagram
print("Building ZXW diagram...")
graph = hamiltonian.build_graph()

print("Graph built successfully")

# Visualize the diagram
print("\nDisplaying ZXW diagram:")
zx.draw(graph)

graph_new = hamiltonian.build_trotter_graph(steps=5, time=1.0)
zx.draw(graph_new)


output_filename = "trotter_export.json"
with open(output_filename, "w") as f:
    f.write(graph_new.to_json())


Building ZXW diagram...


Graph built successfully

Displaying ZXW diagram:


(0.1-0.1j)
0.015198793544007326j
-0.015198793544007326j
0.015198793544007326j
-0.015198793544007326j
-0.005659476635693038j
0.005659476635693038j
(0.1-0.1j)
-0.005659476635693038j
0.005659476635693038j
0.015198793544007326j
-0.015198793544007326j
(0.1-0.1j)
0.015198793544007326j
-0.015198793544007326j
(0.1-0.1j)


In [11]:
# Simplify the diagram
print("Simplifying ZXW diagram...")
hamiltonian._normalize_rows(graph_new)
print("Graph simplified")

# Visualize simplified diagram
print("\nDisplaying simplified ZXW diagram:")
zx.draw(graph_new)


Simplifying ZXW diagram...
Graph simplified

Displaying simplified ZXW diagram:


In [ ]:
qtn = hamiltonian.to_tensor_network()
outer_inds = qtn.outer_inds()
inner_inds = [ind for ind in qtn.ind_map.keys() if ind not in outer_inds]
    
print(f"  Outer indices (inputs/outputs): {len(outer_inds)}")
print(f"  Inner indices (to contract): {len(inner_inds)}")
    

  Outer indices (inputs/outputs): 8
  Inner indices (to contract): 111


In [ ]:
def normalize_graph_rows(graph):
    rows = sorted(set(graph.row(v) for v in graph.vertices()))
    row_mapping = {old_row: new_row for new_row, old_row in enumerate(rows)}
    for v in graph.vertices():
        old_row = graph.row(v)
        new_row = row_mapping[old_row]
        
        if hasattr(graph, '_row') and isinstance(graph._row, dict):
            graph._row[v] = new_row
        
        elif hasattr(graph, 'set_row'):
            graph.set_row(v, new_row)
        else:
            
            try:
                vdata = graph.vertex_data(v)
                if vdata is not None and hasattr(vdata, 'row'):
                    vdata.row = new_row
            except:
                pass
    
    return graph

